# From Once upon a time, to Happily ever after
## A generative journey through Fairyland 

## System set up

In [1]:
import random

import ipywidgets as widgets
import spacy
import torch
from diffusers import StableDiffusionPipeline
from huggingface_hub import notebook_login
from IPython.display import HTML, display
from ipywidgets import fixed, interact_manual
from torch import autocast
from transformers import pipeline

/home/jvieira/sandbox/creativeai/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/jvieira/sandbox/creativeai/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:529: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [2]:
!nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


A [Hugging Face](https://huggingface.co/) account and token is required to use the [Stable Diffusion](https://huggingface.co/CompVis/stable-diffusion-v1-4) text-to-image generator.

In [ ]:
notebook_login()

## Set up image generation

In [3]:
diffusion_device = 0
diffusion_model_path = "stabilityai/stable-diffusion-2"

try:
    del diffusion
except NameError:
    pass

diffusion = StableDiffusionPipeline.from_pretrained(
    diffusion_model_path, use_auth_token=True, revision="fp16", torch_dtype=torch.float16
)
diffusion = diffusion.to(diffusion_device)
print(
    f"Diffusion - device {diffusion_device} | {torch.cuda.memory_allocated(diffusion_device) / 1000000}MiB"
)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW

In [ ]:
def generate_image(prompt: str, num_images_per_prompt: int = 1, width: int = 768, height: int = 768, seed: int = 1024):
    images = diffusion(
        prompt,
        num_images_per_prompt=num_images_per_prompt,
        width=width,
        height=height,
        generator=torch.Generator("cuda").manual_seed(seed),
    )

    return images

In [ ]:
images = generate_image("a photo of steampunk computer", seed=random.randint(0, 100))
for image in images.images:
    display(image)

## Set up text generation

In [4]:
gpt_device = 1
gpt_model_path = "../data/final/grimm_4_1.3B/"

try:
    del gpt
except NameError:
    pass

gpt = pipeline("text-generation", model=gpt_model_path, device=gpt_device)
print(
    f"GPT - device {gpt_device} | {torch.cuda.memory_allocated(gpt_device) / 1000000}MiB"
)

RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW

### Set up text summarisation

Summarisation is used to convert text into image generation prompts.

In [ ]:
summariser_model_path = "facebook/bart-large-cnn"

try:
    del summariser
except NameError:
    pass

summariser = pipeline("summarization", model=summariser_model_path, device=gpt_device)
print(
    f"Summariser - device {gpt_device} | {torch.cuda.memory_allocated(gpt_device) / 1000000}MiB"
)

In [ ]:
def summarise(text: str, min_length: int = 10, max_length: int = 50) -> str:
    if len(text.split()) < max_length:
        return text

    output = summariser(text, min_length=10, max_length=50)
    return output[0]["summary_text"].strip()

## Generation

### Text generation

In [ ]:
text_form = widgets.Output(layout=dict(border="1px solid black", padding="5px"))

text_widget = widgets.Textarea(
    description="Text",
    placeholder="Type a prompt to generate text from",
    layout=dict(width="auto", height="500px"),
)
max_length_widget = widgets.IntText(
    value=250,
    description="Max length",
    tooltip="Maximum number of tokens",
)
temperature_widget = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    description="Temperature",
    tooltip="How sensitive the algorithm is to selecting low probability options",
)
top_k_widget = widgets.IntText(
    value=50,
    description="Top k",
    tooltip="How many potential answers are considered when performing sampling",
)
top_p_widget = widgets.IntText(
    value=1.0,
    min=0.0,
    max=1.0,
    description="Top p",
    tooltip="Min number of tokens are selected where their probabilities add up to top_p",
)
no_repeat_ngram_size_widget = widgets.IntText(
    value=0,
    description="No repeat ngrams",
    tooltip="The size of an n-gram that cannot occur more than once. (0=infinity)",
)

In [ ]:
def generate_text(
    prompt: str,
    max_length: int = 200,
    temperature: float = 0.5,
    top_k: int = 50,
    top_p: float = 1.0,
    no_repeat_ngram_size: int = 0,
) -> str:
    output = gpt(
        prompt,
        max_length=len(prompt) + max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        no_repeat_ngram_size=no_repeat_ngram_size,
    )
    generated_text = output[0]["generated_text"]

    with text_widget.hold_trait_notifications():
        text_widget.value = generated_text


with text_form:
    interact_manual(
        generate_text,
        prompt=text_widget,
        max_length=max_length_widget,
        temperature=temperature_widget,
        top_k=top_k_widget,
        top_p=top_p_widget,
        no_repeat_ngram_size=no_repeat_ngram_size_widget,
    )

display(text_form)

### Image generation

An image is generated for each paragraph in the generated text. If the generated text doesn't have any paragraphs, edit the text in the textarea above and add new lines before running the image generation.

**Need to find a better summariser!**

In [5]:
prompts = []

paragraphs = text_widget.value.split("\n")
for p in paragraphs:
    if p:
        summary = summarise(p)
        prompts.append(summary)


def generate_story(text: str, style: str, seed: int):
    prompts = text.split("\n")

    for idx, prompt in enumerate(prompts):
        seed += idx
        if prompt:
            prompt = f"{prompt} {style}"
            image = generate_image(prompt, seed=seed)[0][0]

            tag = "p"
            display(HTML(f"<{tag}>{paragraphs[idx]}</{tag}>"))
            display(image)
            display(
                HTML(
                    f"<small style='font-size: 8px'>prompt: {prompt}, seed: {seed}</small>"
                )
            )


image_form = widgets.Output(layout=dict(border="1px solid black", padding="5px"))

image_prompt_widget = widgets.Textarea(
    description="Image prompts",
    value="\n\n".join(prompts),
    placeholder="Image prompts, one per line",
    layout=dict(width="auto", height="200px"),
)
image_style_widget = widgets.Text(description="Image style", value="")
image_seed_widget = widgets.IntText(description="Seed", value=random.randint(0, 2048))

with image_form:
    interact_manual(
        generate_story,
        text=image_prompt_widget,
        style=image_style_widget,
        seed=image_seed_widget,
    )

display(image_form)

NameError: name 'text_widget' is not defined